In [3]:
#open the txt file and read object categories.
f = open("paths.txt", "r") 
labels = []
for x in f: # read the txt file line by line
    labels.append(x.split(':')[1]) #add it to the list
print(len(labels))

4


In [ ]:
from torchvision.models import resnet50, ResNet50_Weights
import os

os.environ['TORCH_HOME'] = '/home/robotics/Documents/notebooks' #setting the environment variable
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

/home/robotics/Documents/notebooks


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /home/robotics/Documents/notebooks/hub/checkpoints/resnet50-11ad3fa6.pth


In [ ]:
import traitlets
import cv2
import numpy as np
import pyzed.sl as sl
import math
import numpy as np
import sys
import math
import threading
from traitlets.config.configurable import SingletonConfigurable

# Define a Camera class that inherits from SingletonConfigurable
class Camera(SingletonConfigurable):
    color_value = traitlets.Any() # monitor the color_value variable
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        # Create a InitParameters object and set configuration parameters
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA #VGA(672*376), HD720(1280*720), HD1080 (1920*1080) or ...
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA  # Use ULTRA depth mode
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # Use meter units (for depth measurements)

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS: #Ensure the camera has opened succesfully
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

         # Create and set RuntimeParameters after opening the camera
        self.runtime = sl.RuntimeParameters()

        #flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width,self.height,sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C4, sl.MEM.CPU) 

    def _capture_frames(self): #For data capturing only

        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value = self.image.get_data()
                self.color_value= cv2.cvtColor(self.color_value, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())
                
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread
            self.zed.close()

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera()
camera.start() # start capturing the data


In [11]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import sys
import time
import motors

#initialize the Robot class
robot = motors.MotorsYukon(mecanum=False)

#create widgets for the displaying of the image
display_color = widgets.Image(format='jpeg', width='45%') #determine the width of the color image
display_depth = widgets.Image(format='jpeg', width='45%')  #determine the width of the depth image
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside) #display the widget

#callback function, invoked when traitlets detects the changing of the color image
def func(change):
    #Scaling is necessary for real-time data display.
    scale = 0.1
    resized_image = cv2.resize(change['new'], None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    cv2.circle(resized_image, (int(camera.width*scale//2),int(camera.height*scale//2)), 1, (0, 255, 0))
    display_color.value = bgr8_to_jpeg(resized_image)

    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(camera.depth_image , alpha=0.03), cv2.COLORMAP_JET)
    resized_depth_colormap=cv2.resize(depth_colormap, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    display_depth.value = bgr8_to_jpeg(resized_depth_colormap)

camera.observe(func, names=['color_value'])
i=0
#the following code will classify the image and move the robot
while (i < 100):
    t1 = cv2.getTickCount() 
    imgsized= cv2.resize(camera.color_value,(224,224)) #resize the image
    x = cv2.cvtColor(imgsized, cv2.COLOR_BGR2RGB) #convert to RGB as required by the model
    x = x.transpose((2, 0, 1)) #swith the image channels
    x = torch.from_numpy(x).float() #convert to type float
    mean = 255.0 * np.array([0.485, 0.456, 0.406]) #mean value
    stdev = 255.0 * np.array([0.229, 0.224, 0.225]) # for the nomalization of the input image
    normalize = torchvision.transforms.Normalize(mean, stdev)
    x = normalize(x)
    x = x.to(device)# send the data to GPU device
    x = x[None, ...] # increase the image dimension, the model takes a batch of images and the batch size is 1
    output = model(x) #classfy the images
    predict_id = output.max(1, keepdim=True)[1].item() #get the label
    print(predict_id)
    print(labels[predict_id]," prediction time ", (cv2.getTickCount()-t1)/cv2.getTickFrequency())
    if (predict_id) == "forward":
        robot.forward(0.5) 
    elif(predict_id == 'left'):
        robot.left(0.5) 
    elif(predict_id == 'right'):
        robot.right(0.5) 
    else:
        robot.stop() 


747


IndexError: list index out of range

In [ ]:
camera.stop()
robot.stop()